# Python

In [100]:
import importlib
import numpy as np
import time

In [378]:
from dna_v1 import DNA # To get this to work, needed to put 'if name == main' at bottom of dna.py 

In [374]:
importlib.reload(dna_v1)

<module 'dna_v1' from '/home/mjmrose/workspace/sbox-mjmrose/Bids/genAlgo/dna_v1.py'>

# Testing

In [382]:
tgt = "To be or not to be."
pop_sz = 8
mut_rate = 0.3
fitness_exp = 2
midpt_bool = False
pop = Population(tgt, mut_rate, pop_sz, fitness_exp)

In [383]:
draw(pop, 2)

World Record: 0.05263157894736842
Best: .+`\JM1>=,qC 1@U"ZG
Average: 0.006578947368421052


In [384]:
pop.mating_pool

[(2, 1.0),
 (0, 0.0),
 (1, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0)]

In [385]:
for i in range(len(pop.population)):
    print(pop.population[i].genes, pop.population[i].fitness)

['F', 'b', 'E', '?', 't', 'U', 'f', 'K', ';', '%', 'Y', 'O', 't', ';', 'g', 'c', 'D', 'N', '*'] 0.0
['x', 'F', 'k', '8', '_', 'I', 'C', 'i', 'e', 'w', 'O', '3', '(', 'w', '7', ',', '1', ':', '-'] 0.0
['.', '+', '`', '\\', 'J', 'M', '1', '>', '=', ',', 'q', 'C', ' ', '1', '@', 'U', '"', 'Z', 'G'] 0.0027700831024930744
['Q', '#', '8', "'", 'C', 'U', 'R', 'g', '.', '2', '1', 'i', 'e', 'w', '^', ':', 'D', 'x', 'd'] 0.0
['7', 'M', 't', 'Z', 'N', '1', '.', 'T', 'b', 'z', 'f', '>', 'T', 'l', 'v', 's', 'G', '_', "'"] 0.0
[';', 'e', 'N', '3', '7', 's', '[', 'v', '^', 'M', 'H', '9', 'M', 'e', 'f', '_', '=', '`', 'q'] 0.0
['z', '+', '`', '\\', 'J', 'U', '1', '>', '=', ',', '4', 'C', 'e', '1', '/', 'U', '"', 'Z', 'G'] 0
['.', '+', 'M', '\\', 'o', 'M', '1', ' ', '=', ':', 'o', 'C', ' ', '1', '@', 'U', 'P', 'Z', '*'] 0


# Class Definition

In [415]:
# A class to describe a population of virtual organisms
# In this case, each organism is just an instance of a DNA object

class Population:
    def __init__(self, tgt, mut_rate, pop_sz, fitness_exp, replace_bool = True, midpt_bool = False, verbose=False, debug=False):

        self.population = [] # Array to hold the current population
        self.mating_pool = [] # List which we will use for our "mating pool"
        self.generations = 0 # Number of generations
        self.finished = False # Are we finished evolving?
        self.tgt = tgt # Target phrase
        self.mut_rate = mut_rate # Mutation rate
        self.perfect_score = 1
        self.best = ""
        self.fitness_sum = 0
        self.fitness_exp = fitness_exp
        self.replace_bool = replace_bool
        self.midpt_bool = midpt_bool
        self.verbose = verbose
        self.debug = debug

        for i in range(pop_sz):
            self.population.append( DNA( len(self.tgt), verbose=self.verbose ) )

    def calc_fitness(self):
        '''
        Calculates fitness for every member of the population, exponentiates the fitness and calculates population sum
        '''
        if self.debug: print("start of calc_fitness")
        self.fitness_sum = 0
    
        for i in range( len(self.population) ):
           
            self.population[i].calc_fitness(self.tgt)
            
            self.population[i].fitness = self.population[i].fitness**self.fitness_exp
            
            self.fitness_sum += self.population[i].fitness

    def gen_mating_pool(self):
        if self.debug: print("start of gen_mating_pool")
        self.mating_pool = []
    
        for i in range( len(self.population) ): 
    
            # Appending (idx, normalized fitness) for each mem of pop
            self.mating_pool.append( (i, self.population[i].fitness / max(self.fitness_sum, 1e-4) ) )
    
        # Sorting by fitness score in descending order
        self.mating_pool.sort(reverse=True, key = lambda x : x[1])
        
    
    def pick_mem_from_mating_pool(self):
        '''
        Selects a member (mem) from the mating pool to participate in crossover.

        Steps for selection:
            1). Draw random number b/w 0-1 (val)
            2). Subtract normalized fitness of 1st mem of mating pool (highest fitness) from val
            3). If val is now negative, mem of pop corresponding to first mem of mating pool is selected for crossover.
            4). If val is still positive, move to 2nd mem of mating pool and repeat until val is negative
        '''
        if self.debug: print("Start of pick_mem_from_mating_pool")
            
        val = np.random.random()
        
        if self.verbose: print(f"val: {val}")
        if self.verbose: print(f"mating pool: {self.mating_pool}")
            
        for i in range( len(self.mating_pool) ):
            val -= self.mating_pool[i][1]
            if val < 0:
                break
        if self.verbose: print(f"idx of mating pool: {self.mating_pool[i][0]}")
        return self.mating_pool[i][0]
    

    # Create a new generation -->  Refill the population with children from the mating pool
    def gen_new_pop(self, num_new_mems):
        '''
        Generates new members (mems) of population by probabilistically mating existing mems of the population.
        Mems of the population w/ higher fitness are more likely to be selected for mating.

        Inputs:
            num_new_mems: number of new mems to add to the population or to replace in existing population
            replace_bool: if True, will replace 'num_new_mems' mems of pop w/ lowest fitness w/ new children 
                          if False, new children will be appended to existing population
        '''
        if self.debug: print("Start of gen_new_pop")
        children = []
        
        for i in range( num_new_mems ): 
                
            idx1 = self.pick_mem_from_mating_pool()
            idx2 = self.pick_mem_from_mating_pool()
            
            if idx1 == idx2: # Sampling a random (top 10 fitness) member if idx1 == idx2
                rand_hi_idx = np.random.randint( min(10, len(self.population)) )
                idx2 = self.mating_pool[rand_hi_idx][0]
                if self.verbose: print("idx1 == idx2 dealt with")
            
            partnerA = self.population[idx1]
            partnerB = self.population[idx2]
            
            if self.verbose: print(f"PartnerA: {partnerA.get_phrase()}")
            if self.verbose: print(f"PartnerB: {partnerB.get_phrase()}")
    
            child = partnerA.crossover(partnerB, midpt_bool=self.midpt_bool)
            child.mutate(self.mut_rate)
            if self.verbose: print(f"Child: {child.get_phrase()}")
                
            children.append(child)

        if self.replace_bool:  
            for i in range(len(children)):
                replace_idx = self.mating_pool[len(self.mating_pool) - i - 1][0]
                self.population[replace_idx] = children[i] # Overwrites fitness of self.population[replace_idx] w/ 0
        else:
            self.population.extend(children)

        self.generations += 1


    def get_best(self):
        return self.best

    def evaluate(self):
        '''
        Computes the current "most fit" member of the population and whether the perfect score has been achieved.
        '''
        world_record = 0
        idx = 0
        for i in range(len(self.population)): 
            if self.population[i].fitness > world_record:
                idx = i
                world_record = self.population[i].fitness
                
        print(f"World Record: {world_record**(1/self.fitness_exp)}")

        self.best = self.population[idx].get_phrase()
        if world_record**(1/self.fitness_exp) == self.perfect_score:
            self.finished = True

    def is_finished(self):
        return self.finished

    def get_generations(self):
        return self.generations

    # Compute average fitness for the population
    def get_average_fitness(self):
        total = 0
        for i in range( len( self.population ) ):
            total += self.population[i].fitness**(1/self.fitness_exp)
        return total / len(self.population)

In [416]:
def draw(population, num_new_mems):
    
    # Calculate fitness for each mem of pop, take fitness**fitness_exp and calc fitness sum
    population.calc_fitness()
    
    # Generate mating pool array by sorting normalized fitness values
    population.gen_mating_pool()
    
    # Generate new population mems by crossover b/w existing mems of mating pool
    # Either replace existing mems or add new mems to pop
    population.gen_new_pop(num_new_mems)
    
    # Compute most fit mem of pop and determine if finished
    population.evaluate()
    
    print(f"Best: {population.get_best()}")
    print(f"Average: {population.get_average_fitness()}")

    # If we found the target phrase, stop
    if population.is_finished():
        print("We did it :)")
        print(f"Result: {pop.get_best()}")
        print(f"Num gens: {pop.get_generations()}")

### Initializing a population

In [422]:
tgt = "To be or not to be."
pop_sz = 500
mut_rate = 0.3
fitness_exp = 2
midpt_bool = False
pop = Population(tgt, mut_rate, pop_sz, fitness_exp, midpt_bool = False)

# Testing speed of convergence w/ gene selection based on fitness

In [423]:
start = time.time()
for i in range(1000):
    draw(pop, 100)
    if pop.is_finished():
        break
print(f"\nTime: {time.time() - start}")

World Record: 0.10526315789473684
Best: ?#E^5>o- ;5DX4MY,;)
Average: 0.009157894736842108
World Record: 0.21052631578947367
Best: 5(Ev5Som ;5[ 4M',ef
Average: 0.01926315789473683
World Record: 0.21052631578947367
Best: 5(2*5SC] ;55 4o&,ef
Average: 0.035157894736842055
World Record: 0.21052631578947367
Best: 5(2*5dCz yRR 4o&zef
Average: 0.05305263157894746
World Record: 0.2631578947368421
Best: i#*/ebo- iBZ7]t bgW
Average: 0.07357894736842135
World Record: 0.3157894736842105
Best: 5D bQSU](;r! tlsbef
Average: 0.0860000000000004
World Record: 0.3157894736842105
Best: 5D bQSU](;r! tlsbef
Average: 0.09873684210526341
World Record: 0.3157894736842105
Best: 5D bQSU](;r! tlsbef
Average: 0.11178947368421077
World Record: 0.3684210526315789
Best: yD b:SU( \X! tlsbef
Average: 0.1198947368421052
World Record: 0.3684210526315789
Best: yD bJS%( \:" tlsbef
Average: 0.12873684210526268
World Record: 0.3684210526315789
Best: yD bJS%( \:" tlsbef
Average: 0.13978947368421019
World Record: 0.368421052631

World Record: 0.631578947368421
Best: TDtjC or)"oe to be.
Average: 0.4111578947368384
World Record: 0.631578947368421
Best: TDtjC or)"oe to be.
Average: 0.41294736842104884
World Record: 0.631578947368421
Best: TDtjC or)"oe to be.
Average: 0.4138947368421015
World Record: 0.631578947368421
Best: TDtjC or)"oe to be.
Average: 0.4146315789473647
World Record: 0.631578947368421
Best: TDtjC or)"oe to be.
Average: 0.41578947368420677
World Record: 0.631578947368421
Best: TDtjC or)"oe to be.
Average: 0.416105263157891
World Record: 0.631578947368421
Best: TDtjC or)"oe to be.
Average: 0.4183157894736805
World Record: 0.631578947368421
Best: TDtjC or)"oe to be.
Average: 0.4189473684210489
World Record: 0.631578947368421
Best: TDtjC or)"oe to be.
Average: 0.4191578947368384
World Record: 0.631578947368421
Best: TDtjC or)"oe to be.
Average: 0.41978947368420677
World Record: 0.631578947368421
Best: TDtjC or)"oe to be.
Average: 0.4208421052631542
World Record: 0.631578947368421
Best: TDtjC or)"oe t

World Record: 0.7368421052631579
Best: TV be3or noX5t^ be.
Average: 0.4864210526315756
World Record: 0.7368421052631579
Best: TV be3or noX5t^ be.
Average: 0.4868421052631546
World Record: 0.7368421052631579
Best: TV be3or noX5t^ be.
Average: 0.4875789473684177
World Record: 0.7368421052631579
Best: TV be3or noX5t^ be.
Average: 0.4879999999999967
World Record: 0.7368421052631579
Best: TV be3or noX5t^ be.
Average: 0.48842105263157565
World Record: 0.7368421052631579
Best: TV be3or noX5t^ be.
Average: 0.48863157894736514
World Record: 0.7368421052631579
Best: TV be3or noX5t^ be.
Average: 0.48999999999999677
World Record: 0.7368421052631579
Best: TV be3or noX5t^ be.
Average: 0.4904210526315757
World Record: 0.7368421052631579
Best: TV be3or noX5t^ be.
Average: 0.4909473684210494
World Record: 0.7368421052631579
Best: TV be3or noX5t^ be.
Average: 0.4913684210526284
World Record: 0.7368421052631579
Best: TV be3or noX5t^ be.
Average: 0.49252631578947054
World Record: 0.7368421052631579
Best: 

World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5525263157894715
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5530526315789452
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5536842105263137
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5542105263157873
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5547368421052611
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5555789473684188
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5565263157894716
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5576842105263137
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5585263157894717
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5590526315789452
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5603157894736821
World Record: 0.8947368421052632
Best: To b

World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5856842105263159
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5860000000000001
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5863157894736843
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5865263157894738
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5865263157894738
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5870526315789475
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5872631578947369
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5876842105263158
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5888421052631579
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.5891578947368422
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.59
World Record: 0.8947368421052632
Best: To be or nHt to Me

World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.6099999999999985
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.6099999999999985
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.6099999999999985
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.6099999999999985
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.6102105263157881
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.6103157894736828
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.6104210526315776
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.6105263157894723
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.6106315789473671
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.6109473684210512
World Record: 0.8947368421052632
Best: To be or nHt to Me.
Average: 0.6110526315789461
World Record: 0.8947368421052632
Best: To b

World Record: 0.8947368421052632
Best: Uo be or n8t to be.
Average: 0.6281052631578928
World Record: 0.8947368421052632
Best: Uo be or n8t to be.
Average: 0.6283157894736823
World Record: 0.8947368421052632
Best: Uo be or n8t to be.
Average: 0.6283157894736823
World Record: 0.8947368421052632
Best: Uo be or n8t to be.
Average: 0.6285263157894717
World Record: 0.8947368421052632
Best: Uo be or n8t to be.
Average: 0.6285263157894717
World Record: 0.8947368421052632
Best: Uo be or n8t to be.
Average: 0.6286315789473664
World Record: 0.8947368421052632
Best: Uo be or n8t to be.
Average: 0.6286315789473664
World Record: 0.8947368421052632
Best: Uo be or n8t to be.
Average: 0.6286315789473664
World Record: 0.8947368421052632
Best: Uo be or n8t to be.
Average: 0.6290526315789452
World Record: 0.8947368421052632
Best: Uo be or n8t to be.
Average: 0.6292631578947347
World Record: 0.8947368421052632
Best: Uo be or n8t to be.
Average: 0.6294736842105241
World Record: 0.8947368421052632
Best: Uo b

World Record: 0.8947368421052632
Best: To b9 or noO to be.
Average: 0.643052631578946
World Record: 0.8947368421052632
Best: To b9 or noO to be.
Average: 0.6432631578947356
World Record: 0.8947368421052632
Best: To b9 or noO to be.
Average: 0.6433684210526305
World Record: 0.8947368421052632
Best: To b9 or noO to be.
Average: 0.6433684210526305
World Record: 0.8947368421052632
Best: To b9 or noO to be.
Average: 0.6434736842105252
World Record: 0.8947368421052632
Best: To b9 or noO to be.
Average: 0.6436842105263147
World Record: 0.8947368421052632
Best: To b9 or noO to be.
Average: 0.6436842105263147
World Record: 0.8947368421052632
Best: To b9 or noO to be.
Average: 0.6436842105263147
World Record: 0.8947368421052632
Best: To b9 or noO to be.
Average: 0.6437894736842095
World Record: 0.8947368421052632
Best: To b9 or noO to be.
Average: 0.6437894736842095
World Record: 0.8947368421052632
Best: To b9 or noO to be.
Average: 0.6437894736842095
World Record: 0.8947368421052632
Best: To b9

World Record: 0.9473684210526315
Best: To be or no$ to be.
Average: 0.6521052631578959
World Record: 0.9473684210526315
Best: To be or no$ to be.
Average: 0.6521052631578959
World Record: 0.9473684210526315
Best: To be or no$ to be.
Average: 0.6521052631578959
World Record: 0.9473684210526315
Best: To be or no$ to be.
Average: 0.6523157894736852
World Record: 0.9473684210526315
Best: To be or no$ to be.
Average: 0.65242105263158
World Record: 0.9473684210526315
Best: To be or no$ to be.
Average: 0.6525263157894746
World Record: 0.9473684210526315
Best: To be or no$ to be.
Average: 0.6526315789473695
World Record: 0.9473684210526315
Best: To be or no$ to be.
Average: 0.6530526315789484
World Record: 0.9473684210526315
Best: To be or no$ to be.
Average: 0.6530526315789484
World Record: 0.9473684210526315
Best: To be or no$ to be.
Average: 0.6531578947368433
World Record: 0.9473684210526315
Best: To be or no$ to be.
Average: 0.6531578947368433
World Record: 0.9473684210526315
Best: To be 

World Record: 0.9473684210526315
Best: To be or not toYbe.
Average: 0.6632631578947384
World Record: 0.9473684210526315
Best: To be or not toYbe.
Average: 0.6633684210526333
World Record: 0.9473684210526315
Best: To be or not toYbe.
Average: 0.6633684210526333
World Record: 0.9473684210526315
Best: To be or not toYbe.
Average: 0.6633684210526333
World Record: 0.9473684210526315
Best: To be or not toYbe.
Average: 0.6633684210526333
World Record: 1.0
Best: To be or not to be.
Average: 0.6640000000000017
We did it :)
Result: To be or not to be.
Num gens: 902

Time: 15.831934213638306


# Testing midpt

In [410]:
tgt = "To be or not to be."
pop_sz = 500
mut_rate = 0.3
fitness_exp = 2
midpt_bool = False
pop = Population(tgt, mut_rate, pop_sz, fitness_exp)

True


In [411]:
start = time.time()
for i in range(1000):
    draw(pop, 100)
    if pop.is_finished():
        break
print(f"\nTime: {time.time() - start}")

here
World Record: 0.10526315789473684
Best: Tp W;QSo7qwyTDOSc@'
Average: 0.01084210526315789
here
World Record: 0.15789473684210525
Best: d%@dexnq^DNN6ToIgZ.
Average: 0.02115789473684207
here
World Record: 0.15789473684210525
Best: Te C-[S&"n7!\fPq5@'
Average: 0.033894736842105186
here
World Record: 0.21052631578947367
Best: Ao ,!ovqti]>))og<".
Average: 0.04831578947368431
here
World Record: 0.2631578947368421
Best: T9 U&jD/ $NY 9$mbUI
Average: 0.06473684210526341
here
World Record: 0.2631578947368421
Best: T%Xfe`S  8">MToIaj.
Average: 0.07431578947368452
here
World Record: 0.2631578947368421
Best: T%Xfe`S  8">MToIaj.
Average: 0.08536842105263175
here
World Record: 0.3157894736842105
Best: T%Ufe,o? 8S>sToIa?.
Average: 0.09778947368421075
here
World Record: 0.3157894736842105
Best: T%Ufe,o? 8S>sToIa?.
Average: 0.10821052631578965
here
World Record: 0.3157894736842105
Best: Ty+Ge+*F n"pIf;1bM.
Average: 0.11715789473684206
here
World Record: 0.3157894736842105
Best: Ty+Ge+*F n"pIf;1bM.
A

here
World Record: 0.631578947368421
Best: ToBVeio2 nkt)to beP
Average: 0.4031578947368418
here
World Record: 0.631578947368421
Best: ToBVeio2 nkt)to beP
Average: 0.40389473684210475
here
World Record: 0.631578947368421
Best: ToBVeio2 nkt)to beP
Average: 0.4047368421052624
here
World Record: 0.631578947368421
Best: ToBVeio2 nkt)to beP
Average: 0.4051578947368412
here
World Record: 0.6842105263157895
Best: To 4e oP >Ht;t) be.
Average: 0.40673684210526195
here
World Record: 0.6842105263157895
Best: To 4e oP >Ht;t) be.
Average: 0.40705263157894606
here
World Record: 0.6842105263157895
Best: To 4e oP >Ht;t) be.
Average: 0.4076842105263142
here
World Record: 0.6842105263157895
Best: 4o be>or !ot Qo (es
Average: 0.4088421052631561
here
World Record: 0.6842105263157895
Best: 4o be>or !ot Qo (es
Average: 0.4102105263157872
here
World Record: 0.6842105263157895
Best: 4o be>or !ot Qo (es
Average: 0.4114736842105235
here
World Record: 0.6842105263157895
Best: 4o be>or !ot Qo (es
Average: 0.412421

here
World Record: 0.7368421052631579
Best: TF bY 8r Aotfto be.
Average: 0.4950526315789441
here
World Record: 0.7368421052631579
Best: TF bY 8r Aotfto be.
Average: 0.4955789473684178
here
World Record: 0.7368421052631579
Best: TF bY 8r Aotfto be.
Average: 0.4961052631578915
here
World Record: 0.7368421052631579
Best: TF bY 8r Aotfto be.
Average: 0.4965263157894705
here
World Record: 0.7368421052631579
Best: TF bY 8r Aotfto be.
Average: 0.4967368421052599
here
World Record: 0.7368421052631579
Best: TF bY 8r Aotfto be.
Average: 0.4974736842105231
here
World Record: 0.7368421052631579
Best: TF bY 8r Aotfto be.
Average: 0.49778947368420734
here
World Record: 0.7368421052631579
Best: TF bY 8r Aotfto be.
Average: 0.49831578947368105
here
World Record: 0.7368421052631579
Best: TF bY 8r Aotfto be.
Average: 0.4984210526315757
here
World Record: 0.7368421052631579
Best: TF bY 8r Aotfto be.
Average: 0.4986315789473652
here
World Record: 0.7368421052631579
Best: TF bY 8r Aotfto be.
Average: 0.499

here
World Record: 0.8421052631578947
Best: jq be oZ not to be.
Average: 0.5439999999999974
here
World Record: 0.8421052631578947
Best: jq be oZ not to be.
Average: 0.5445263157894712
here
World Record: 0.8421052631578947
Best: jq be oZ not to be.
Average: 0.5448421052631554
here
World Record: 0.8421052631578947
Best: jq be oZ not to be.
Average: 0.54494736842105
here
World Record: 0.8421052631578947
Best: jq be oZ not to be.
Average: 0.5459999999999974
here
World Record: 0.8421052631578947
Best: jq be oZ not to be.
Average: 0.5463157894736816
here
World Record: 0.8421052631578947
Best: jq be oZ not to be.
Average: 0.5468421052631554
here
World Record: 0.8421052631578947
Best: jq be oZ not to be.
Average: 0.5475789473684185
here
World Record: 0.8421052631578947
Best: jq be oZ not to be.
Average: 0.5481052631578922
here
World Record: 0.8421052631578947
Best: jq be oZ not to be.
Average: 0.5483157894736816
here
World Record: 0.8421052631578947
Best: jq be oZ not to be.
Average: 0.5491578

here
World Record: 0.8421052631578947
Best: To bY or notnuo be.
Average: 0.5753684210526311
here
World Record: 0.8421052631578947
Best: To bY or notnuo be.
Average: 0.5757894736842101
here
World Record: 0.8421052631578947
Best: To bY or notnuo be.
Average: 0.5757894736842101
here
World Record: 0.8421052631578947
Best: To bY or notnuo be.
Average: 0.5761052631578942
here
World Record: 0.8421052631578947
Best: To bY or notnuo be.
Average: 0.5761052631578942
here
World Record: 0.8421052631578947
Best: To bY or notnuo be.
Average: 0.5764210526315785
here
World Record: 0.8421052631578947
Best: To bY or notnuo be.
Average: 0.5771578947368418
here
World Record: 0.8421052631578947
Best: To bY or notnuo be.
Average: 0.5775789473684207
here
World Record: 0.8421052631578947
Best: To bY or notnuo be.
Average: 0.5785263157894734
here
World Record: 0.8421052631578947
Best: To bY or notnuo be.
Average: 0.5788421052631576
here
World Record: 0.8421052631578947
Best: To bY or notnuo be.
Average: 0.57894

...

here
World Record: 0.8947368421052632
Best: To be or notito b].
Average: 0.6164210526315774
here
World Record: 0.8947368421052632
Best: To be or notito b].
Average: 0.616631578947367
here
World Record: 0.8947368421052632
Best: To be or notito b].
Average: 0.616631578947367
here
World Record: 0.8947368421052632
Best: To be or notito b].
Average: 0.616631578947367
here
World Record: 0.8947368421052632
Best: To be or notito b].
Average: 0.6168421052631564
here
World Record: 0.8947368421052632
Best: To be or notito b].
Average: 0.6171578947368407
here
World Record: 0.8947368421052632
Best: To be or notito b].
Average: 0.6172631578947354
here
World Record: 0.8947368421052632
Best: To be or notito b].
Average: 0.6172631578947354
here
World Record: 0.8947368421052632
Best: To be or notito b].
Average: 0.6174736842105248
here
World Record: 0.8947368421052632
Best: To be or notito b].
Average: 0.6175789473684197
here
World Record: 0.8947368421052632
Best: To be orpnot ,o be.
Average: 0.61831578

here
World Record: 0.8947368421052632
Best: Fo be or not tZ be.
Average: 0.6376842105263129
here
World Record: 0.8947368421052632
Best: Fo be or not tZ be.
Average: 0.6376842105263129
here
World Record: 0.8947368421052632
Best: Fo be or not tZ be.
Average: 0.6378947368421024
here
World Record: 0.8947368421052632
Best: Fo be or not tZ be.
Average: 0.6383157894736813
here
World Record: 0.8947368421052632
Best: Fo be or not tZ be.
Average: 0.638421052631576
here
World Record: 0.8947368421052632
Best: Fo be or not tZ be.
Average: 0.6386315789473654
here
World Record: 0.8947368421052632
Best: Fo be or not tZ be.
Average: 0.6387368421052603
here
World Record: 0.8947368421052632
Best: Fo be or not tZ be.
Average: 0.6391578947368391
here
World Record: 0.8947368421052632
Best: Fo be or not tZ be.
Average: 0.6393684210526286
here
World Record: 0.8947368421052632
Best: Fo be or not tZ be.
Average: 0.6395789473684182
here
World Record: 0.8947368421052632
Best: Fo be or not tZ be.
Average: 0.640105

here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6455789473684199
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6455789473684199
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6458947368421042
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6461052631578937
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6462105263157886
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6462105263157886
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6463157894736834
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6463157894736834
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6464210526315781
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6466315789473678
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.64663

here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.654842105263159
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6550526315789486
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6550526315789486
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.655263157894738
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.655263157894738
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.655263157894738
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6554736842105275
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6554736842105275
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6555789473684221
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6555789473684221
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.655894736

here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6665263157894753
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6665263157894753
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.66663157894737
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6667368421052648
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6668421052631597
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6668421052631597
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6668421052631597
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6673684210526333
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6673684210526333
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6673684210526333
here
World Record: 0.9473684210526315
Best: To Ee or not to be.
Average: 0.6677894

In [409]:
len(pop.population)

500

# Debugging

In [ ]:
draw(pop, 5, True)

In [ ]:
for i in range(len(pop.population)):
    print(pop.population[i].fitness, pop.population[i].get_phrase())



In [ ]:
pop.mating_pool

In [ ]:
for i in range(len(pop.population)):
    print(pop.population[i].fitness, pop.population[i].get_phrase())


In [ ]:

pop.mating_pool